---
file_format: mystnb
kernelspec:
  name: python3
title:      LOST in Triangulation
date:       2023-02-04
authors:
  - name:   Akshay Krishnan
    url:    https://akshay-krishnan.github.io/
  - name:   Sebastien Henry
    url:    https://seal.ae.gatech.edu/node/14
  - name:   Frank Dellaert
    url:    https://dellaert.github.io/
  - name:   John Christian
    url:    https://seal.ae.gatech.edu/node/1

---


This post introduces the triangulation problem and some commonly used solutions (both optimal and sub-optimal), along with GTSAM code examples. We also discuss a recently proposed **linear optimal sine triangulation method (LOST)** and compare its performance to other methods. 

:::{admonition} TL;DR:

LOST can be easily enabled in GTSAM by setting `useLOST=True` in `triangulatePoint3` to get state-of-the-art linear triangulation!
:::

# The triangulation problem

**Triangulation** is the problem of estimating the location of a 3D point from multiple direction measurements. The problem is commonly encountered in the domains of 3D reconstruction, robot localization, and spacecraft navigation, amongst others. The "direction measurements" are usually obtained from 2D observations of the same 3D point in different cameras.

## A simple solution: the Direct Linear Transform

A commonly used method for triangulation is the **Direct Linear Transform (DLT)**, which poses the estimation of the 3D point as a least squares problem. Given the position of camera $i$ in the world frame as $\mathbf{^wp_i}$ and the 2D measurement (in homogeneous coordinates) of the point in this camera as $\mathbf{x_i}$, it uses the following constraint:

```{math}
:label: dlt-constraint
 \mathbf{x_i} \times \mathbf{R^i_w} (\mathbf{^wr} - \mathbf{^wp_i})  &= \mathbf{0} \\

\text{i.e } [\mathbf{x_i} \times] \mathbf{R^i_w} (\mathbf{^wr} - \mathbf{^wp_i})  &= \mathbf{0}
```

Here: 
- $\mathbf{^wr}$ is the position of the 3D point in the world frame, which is to be estimated
- $[\mathbf{x_i} \times]$ is a skew-symmetric matrix with elements $\mathbf{x_i}$, such that $[\mathbf{x_i} \times] \ \mathbf{b} = \mathbf{x_i} \times \mathbf{b}$ 
- $\mathbf{R^i_w} \in SO(3)$ is the rotation from world to camera frame $i$

{eq}`dlt-constraint` can be expressed in terms of the pixel measurements $\mathbf{u_i} = \mathbf{K_i} \mathbf{x_i}$ using the intrinsic calibration matrix of the camera $\mathbf{K_i}$:

```{math}
    [\mathbf{K_i^{-1}} \mathbf{u_i} \times] \mathbf{R^i_w} (\mathbf{^wr} - \mathbf{^wp_i})  = \mathbf{0}
```

With at least 2 camera measurements, this can be expressed as a least-squares problem of the form $\mathbf{A x} = \mathbf{b}$ with a unique solution (except in some degenerate conditions). For $n$ measurements, this equation is:

$$
\begin{equation}
\left[\begin{array}{c}
\left[\mathbf{K_1^{-1}} \mathbf{u_1} \times\right] \mathbf{R^1_w} \cr	\\
\left[\mathbf{K_2^{-1}} \mathbf{u_2} \times\right] \mathbf{R^2_w} \cr	\\
... \cr \\
\left[\mathbf{K_n^{-1}} \mathbf{u_n} \times\right] \mathbf{R^n_w} \cr
\end{array}\right] \mathbf{^wr} = 
\left[\begin{array}{c}
\left[\mathbf{K_1^{-1}} \mathbf{u_1} \times\right] \mathbf{R^1_w} \mathbf{^wp_1} \cr	\\
\left[\mathbf{K_2^{-1}} \mathbf{u_2} \times\right] \mathbf{R^2_w} \mathbf{^wp_2} \cr	\\
... \cr \\
\left[\mathbf{K_n^{-1}} \mathbf{u_n} \times\right] \mathbf{R^n_w} \mathbf{^wp_n} \cr
\end{array}\right]
\end{equation}
$$ (least-squares)


DLT solves the above equation using standard least-squares techniques. It also has interesting connections to the trigonometric sine rule, as discussed in the [LOST paper](https://doi.org/10.2514/1.G006989) ([Arxiv](https://arxiv.org/pdf/2205.12197.pdf)).

The `triangulatePoint3` function in GTSAM provides a convenient way to triangulate points using DLT. Let's look at an example of triangulating a point from noisy measurements. 

```python
import gtsam
import numpy as np
from gtsam import Pose3, Rot3, Point3

# Define parameters for 2 cameras, with shared intrinsics.
pose1 = Pose3()
pose2 = Pose3(Rot3(), Point3(5., 0., -5.))
intrinsics = gtsam.Cal3_S2()
camera1 = gtsam.PinholeCameraCal3_S2(pose1, intrinsics)
camera2 = gtsam.PinholeCameraCal3_S2(pose2, intrinsics)
cameras = gtsam.CameraSetCal3_S2([camera1, camera2])

# Define a 3D point, generate measurements by projecting it to the 
# cameras and adding some noise.
landmark = Point3(0.1, 0.1, 1.5)
m1_noisy = cameras[0].project(landmark) + gtsam.Point2(0.00817, 0.00977)
m2_noisy = cameras[1].project(landmark) + gtsam.Point2(-0.00610, 0.01969)
measurements = gtsam.Point2Vector([m1_noisy, m2_noisy])

# Triangulate!
dlt_estimate = gtsam.triangulatePoint3(cameras, measurements, rank_tol=1e-9, optimize=False)
print("DLT estimation error: {:.04f}".format(np.linalg.norm(dlt_estimate - landmark)))
```

```sh
DLT estimation error: 0.0832
```

Here is an interactive visualization of the DLT estimate, the two camera frustums, and the back-projected rays. Note the difference between the ground truth and the triangulated point. We shall revisit this example below, and compare it against the result from other methods.

In [5]:
import plotly.graph_objects as go

# Define your data
data = [
    go.Scatter3d(
        x=[0.0, -0.8, None, 0.0, -0.8, None, 0.0, 0.8, None, 0.0, 0.8, None, -0.8, -0.8, None, -0.8, 0.8, None, 0.8, -0.8, None, 0.8, 0.8, None],
        y=[0.0, -0.6, None, 0.0, 0.6, None, 0.0, -0.6, None, 0.0, 0.6, None, -0.6, 0.6, None, 0.6, 0.6, None, -0.6, -0.6, None, 0.6, -0.6, None],
        z=[0.0, 1.0, None, 0.0, 1.0, None, 0.0, 1.0, None, 0.0, 1.0, None, 1.0, 1.0, None, 1.0, 1.0, None, 1.0, 1.0, None, 1.0, 1.0, None],
        mode="lines",
        name="camera 1"
    ),
    go.Scatter3d(
        x=[5.0, 4.2, None, 5.0, 4.2, None, 5.0, 5.8, None, 5.0, 5.8, None, 4.2, 4.2, None, 4.2, 5.8, None, 5.8, 4.2, None, 5.8, 5.8, None],
        y=[0.0, -0.6, None, 0.0, 0.6, None, 0.0, -0.6, None, 0.0, 0.6, None, -0.6, 0.6, None, 0.6, 0.6, None, -0.6, -0.6, None, 0.6, -0.6, None],
        z=[-5.0, -4.0, None, -5.0, -4.0, None, -5.0, -4.0, None, -5.0, -4.0, None, -4.0, -4.0, None, -4.0, -4.0, None, -4.0, -4.0, None, -4.0, -4.0, None],
        mode="lines",
        name="camera 2"
    ),
    go.Scatter3d(
        x=[0.1],
        y=[0.1],
        z=[1.5],
        mode="markers",
        name="GT point",
        marker=dict(size=2)
    ),
    go.Scatter3d(
        x=[0.1023714151218403],
        y=[0.16890260533047632],
        z=[1.453409909325872],
        mode="markers",
        name="DLT estimate",
        marker=dict(size=2)
    ),
    go.Scatter3d(
        x=[0.10783485812100543],
        y=[0.11608849005416458],
        z=[1.444684619571376],
        mode="markers",
        name="LOST estimate",
        marker=dict(size=2)
    ),
    go.Scatter3d(
        x=[0.0, 0.14967333333333332, None],
        y=[0.0, 0.15287333333333333, None],
        z=[0.0, 2.0, None],
        mode="lines+markers",
        name="ray 1",
        marker=dict(size=[0, 6, 0], symbol="diamond")
    ),
    go.Scatter3d(
        x=[5.0, -0.3196230769230777, None],
        y=[0.0, 0.2455223076923077, None],
        z=[-5.0, 2.0, None],
        mode="lines+markers",
        name="ray 2",
        marker=dict(size=[0, 6, 0], symbol="diamond")
    )
]

# Define layout
layout = go.Layout(
    scene=dict(
        xaxis=dict(backgroundcolor="#E5ECF6", gridcolor="white", gridwidth=2, linecolor="white", showbackground=True, ticks="", zerolinecolor="white"),
        yaxis=dict(backgroundcolor="#E5ECF6", gridcolor="white", gridwidth=2, linecolor="white", showbackground=True, ticks="", zerolinecolor="white"),
        zaxis=dict(backgroundcolor="#E5ECF6", gridcolor="white", gridwidth=2, linecolor="white", showbackground=True, ticks="", zerolinecolor="white")
    ),
    margin=dict(l=0, r=0, t=0, b=0),
    title=dict(x=0.05),
    hovermode="closest",
)

# Create figure
fig = go.Figure(data=data, layout=layout)

# Show figure
fig.show()